In [11]:
import torch
import torch.nn as nn
from typing import Optional, Tuple

class _ConvolutionModule(torch.nn.Module):
    r"""LE_Conformer convolution module.

    Args:
        input_dim (int): input dimension.
        num_channels (int): number of depthwise convolution layer input channels.
        depthwise_kernel_size (int): kernel size of depthwise convolution layer.
        dropout (float, optional): dropout probability. (Default: 0.0)
        bias (bool, optional): indicates whether to add bias term to each convolution layer. (Default: ``False``)
        use_group_norm (bool, optional): use GroupNorm rather than BatchNorm. (Default: ``False``)
    """

    def __init__(
        self,
        input_dim: int,  # 入力の次元数
        num_channels: int,  # 畳み込み層のチャンネル数
        depthwise_kernel_size: int,  # 深さ方向の畳み込みのカーネルサイズ
        dropout: float = 0.0,  # ドロップアウトの割合
        bias: bool = False,  # バイアス項を使用するかどうか
        use_group_norm: bool = False,  # GroupNormを使用するかどうか
    ) -> None:
        super().__init__()
        # depthwise_kernel_sizeが奇数でなければならない理由は、'SAME'パディングを実現するためです。
        if (depthwise_kernel_size - 1) % 2 != 0:
            raise ValueError("depthwise_kernel_size must be odd to achieve 'SAME' padding.")
        # LayerNorm層
        self.layer_norm = torch.nn.LayerNorm(input_dim)
        # 畳み込み層と活性化関数を含むシーケンシャルなネットワーク
        self.sequential = torch.nn.Sequential(
            # Pointwise convolution: 入力次元から2倍のチャンネル数へ
            torch.nn.Conv1d(
                input_dim,
                2 * num_channels,
                kernel_size=1, # カーネルサイズ1はPointwiseConvの特徴
                stride=1,
                padding=0,
                bias=bias,
            ),
            # GLU活性化関数: チャンネル数を半分にする
            torch.nn.GLU(dim=1),
            # Depthwise convolution: 各チャンネル内での畳み込みを行う
            torch.nn.Conv1d(
                num_channels,
                num_channels,
                depthwise_kernel_size,
                stride=1,
                padding=(depthwise_kernel_size - 1) // 2,
                groups=num_channels,
                bias=bias,
            ),
            # GroupNormまたはBatchNorm層
            torch.nn.GroupNorm(num_groups=1, num_channels=num_channels)
            if use_group_norm
            else torch.nn.BatchNorm1d(num_channels),
            # SiLU活性化関数
            torch.nn.SiLU(),
            # Pointwise convolution: チャンネル数を元の入力次元に戻す
            torch.nn.Conv1d(
                num_channels,
                input_dim,
                kernel_size=1, # カーネルサイズ1はPointwiseConvの特徴
                stride=1,
                padding=0,
                bias=bias,
            ),
            # ドロップアウト層
            torch.nn.Dropout(dropout),
        )

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        r"""
        Args:
            input (torch.Tensor): with shape `(L,B, D)`.

        Returns:
            torch.Tensor: output, with shape `(L,B, D)`.
        """
        x = self.layer_norm(input)
        x = x.permute(1, 2,0) # 1D Convはlengthで畳み込みたい、Dはチャネルとしたいので(B,D,L)とする。
        x = self.sequential(x)
        return x.permute(2,0,1) #(L,B, D)に戻す。
class SEBlock1D(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock1D, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class Conv1DBlockWithSE(nn.Module):
    def __init__(self, in_channels, depthwise_kernel_size):
        super().__init__()
        self.layer_norm = torch.nn.LayerNorm(in_channels)
        self.conv = nn.Conv1d(in_channels, in_channels, 
                depthwise_kernel_size,
                stride=1,
                padding=(depthwise_kernel_size - 1) // 2,)
        self.bn = nn.BatchNorm1d(in_channels)
        self.swish = nn.SiLU()
        self.se = SEBlock1D(in_channels)

    def forward(self, x):
        x = self.layer_norm(x)
        x = x.permute(1, 2, 0)
        x = self.conv(x)
        x = self.bn(x)
        x = self.swish(x)
        x = self.se(x)
        x = x.permute(2, 0, 1)
        return x


class _FeedForwardModule(torch.nn.Module):# こちらは特に迷うこともなく論文そのままなのでコメントは割愛
    r"""Positionwise feed forward layer.

    Args:
        input_dim (int): input dimension.
        hidden_dim (int): hidden dimension.
        dropout (float, optional): dropout probability. (Default: 0.0)
    """

    def __init__(self, input_dim: int, hidden_dim: int, depthwise_kernel_size: int, dropout: float = 0.0) -> None:
        super().__init__()
        self.layer_norm = nn.LayerNorm(input_dim)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.activation = Conv1DBlockWithSE(hidden_dim,depthwise_kernel_size)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, input_dim)

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        input = self.layer_norm(input)
        hidden = self.fc1(input)
        hidden = self.activation(hidden)
        hidden = self.dropout(hidden)
        output = self.fc2(hidden)
        return output
    

In [12]:
class LE_ConformerLayer(torch.nn.Module):
    r"""LE_Conformer layer that constitutes LE_Conformer.

    Args:
        input_dim (int): input dimension.
        ffn_dim (int): hidden layer dimension of feedforward network.
        num_attention_heads (int): number of attention heads.
        depthwise_conv_kernel_size (int): kernel size of depthwise convolution layer.
        dropout (float, optional): dropout probability. (Default: 0.0)
        use_group_norm (bool, optional): use ``GroupNorm`` rather than ``BatchNorm1d``
            in the convolution module. (Default: ``False``)
        convolution_first (bool, optional): apply the convolution module ahead of
            the attention module. (Default: ``False``)
    """

    def __init__(
        self,
        input_dim: int,
        ffn_dim: int,
        num_attention_heads: int,
        depthwise_conv_kernel_size: int,
        dropout: float = 0.0,
        use_group_norm: bool = False,
        convolution_first: bool = False,
    ) -> None:
        super().__init__()

        # 最初のフィードフォワードネットワークを定義します。このネットワークは、入力データを処理し、
        # 隠れ層の次元をffn_dimに変換します。
        self.ffn1 = _FeedForwardModule(input_dim, ffn_dim, depthwise_conv_kernel_size, dropout=dropout)

        # レイヤーノーマライゼーションを定義します。これは、入力データのスケールを正規化するために使用されます。
        self.self_attn_layer_norm = torch.nn.LayerNorm(input_dim)
        # セルフアテンションメカニズムを定義します。これは、入力データの各部分が他の部分とどの程度関連しているかを学習します。
        self.self_attn = torch.nn.MultiheadAttention(input_dim, num_attention_heads, dropout=dropout)
        # ドロップアウトを定義します。これは、ネットワークの過学習を防ぐために使用されます。
        self.self_attn_dropout = torch.nn.Dropout(dropout)

        # 畳み込みモジュールを定義します。これは、入力データに畳み込みを適用し、特徴を抽出します。
        self.conv_module = _ConvolutionModule(
            input_dim=input_dim,
            num_channels=input_dim,
            depthwise_kernel_size=depthwise_conv_kernel_size,
            dropout=dropout,
            bias=True,
            use_group_norm=use_group_norm,
        )

        # 二番目のフィードフォワードネットワークを定義します。このネットワークは、畳み込みモジュールとセルフアテンションメカニズムによって
        # 処理し、出力の次元をinput_dimに戻します。
        self.ffn2 = _FeedForwardModule(input_dim, ffn_dim, depthwise_conv_kernel_size, dropout=dropout)
        # 最終的なレイヤーノーマライゼーションを定義します。これは、出力データのスケールを正規化するために使用されます。
        self.final_layer_norm = torch.nn.LayerNorm(input_dim)
        # 畳み込みモジュールをセルフアテンションメカニズムの前に適用するかどうかを決定します。
        self.convolution_first = convolution_first

    def _apply_convolution(self, input: torch.Tensor) -> torch.Tensor:
        # 入力データを保存します。これは後で残差接続を作成するために使用されます。
        residual = input
        # 入力データの次元を変更します。これは、畳み込みモジュールが適切に機能するために必要です。
        input = input.transpose(0, 1)
        # 畳み込みモジュールを適用します。これにより、入力データに畳み込みが適用され、特徴が抽出されます。
        input = self.conv_module(input)
        # 入力データの次元を元に戻します。これは、後続の処理ステップが適切に機能するために必要です。
        input = input.transpose(0, 1)
        # 残差接続を作成します。これは、ネットワークが入力データの重要な情報を保持できるようにするために使用されます。
        input = residual + input
        return input

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        r"""
        Args:
            input (torch.Tensor): input, with shape `(T, B, D)`.
            key_padding_mask (torch.Tensor or None): key padding mask to use in self attention layer.

        Returns:
            torch.Tensor: output, with shape `(T, B, D)`.
        """
        # 入力データを保存します。これは後で残差接続を作成するために使用されます。
        residual = input
        # 最初のフィードフォワードネットワークを適用します。これにより、入力データが処理され、隠れ層の次元がffn_dimに変換されます。
        x = self.ffn1(input)
        # スケールされた残差接続を作成します。これは、ネットワークが入力データの重要な情報を保持できるようにするために使用されます。
        x = x * 0.5 + residual

        # 畳み込みモジュールを最初に適用する場合、ここでそれを行います。
        if self.convolution_first:
            x = self._apply_convolution(x)

        # 入力データを保存します。これは後で残差接続を作成するために使用されます。
        residual = x
        # レイヤーノーマライゼーションを適用します。これは、入力データのスケールを正規化するために使用されます。
        x = self.self_attn_layer_norm(x)
        # セルフアテンションメカニズムを適用します。これは、入力データの各部分が他の部分とどの程度関連しているかを学習します。
        x, _ = self.self_attn(
            query=x,
            key=x,
            value=x,
            need_weights=False,
            
        )
        # ドロップアウトを適用します。これは、ネットワークの過学習を防ぐために使用されます。
        x = self.self_attn_dropout(x)
        # 残差接続を作成します。これは、ネットワークが入力データの重要な情報を保持できるようにするために使用されます。
        x = x + residual

        # 畳み込みモジュールを最初に適用しなかった場合、ここでそれを行います。
        if not self.convolution_first:
            x = self._apply_convolution(x)

        # 入力データを保存します。これは後で残差接続を作成するために使用されます。
        residual = x
        # 二番目のフィードフォワードネットワークを適用します。これにより、畳み込みモジュールとセルフアテンションメカニズムによって
        # 処理されたデータが処理され、出力の次元がinput_dimに戻されます。
        x = self.ffn2(x)
        # スケールされた残差接続を作成します。これは、ネットワークが入力データの重要な情報を保持できるようにするために使用されます。
        x = x * 0.5 + residual

        # 最終的なレイヤーノーマライゼーションを適用します。これは、出力データのスケールを正規化するために使用されます。
        x = self.final_layer_norm(x)
        # 処理されたデータを返します。
        return x


In [13]:
class LE_Conformer(torch.nn.Module):
    def __init__(
        self,
        input_dim: int,
        num_heads: int,
        ffn_dim: int,
        num_layers: int,
        depthwise_conv_kernel_size: int,
        output_dim,
        dropout: float = 0.0,
        use_group_norm: bool = False,
        convolution_first: bool = False,
    ):
        super().__init__()

        self.LE_conformer_layers = torch.nn.ModuleList(
            [
                LE_ConformerLayer(
                    input_dim,
                    ffn_dim,
                    num_heads,
                    depthwise_conv_kernel_size,
                    dropout=dropout,
                    use_group_norm=use_group_norm,
                    convolution_first=convolution_first,
                )
                for _ in range(num_layers)
            ]
        )
        self.li = 

    def forward(self, input: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:


        x = input
        for layer in self.LE_conformer_layers:
            x = layer(x)
        return x

In [2]:
# LE_Conformerのインスタンスを作成
model = LE_Conformer(
    input_dim=32,  # 入力の次元数
    num_heads=8,  # MultiheadAttentionのヘッド数
    ffn_dim=128,  # FeedForwardネットワークの隠れ層の次元数
    num_layers=6,  # LE_ConformerLayerの数
    depthwise_conv_kernel_size=3,  # Depthwise convolutionのカーネルサイズ
    dropout=0.1,  # ドロップアウトの割合
    use_group_norm=False,  # GroupNormを使用するかどうか
    convolution_first=False  # 畳み込みを最初に行うかどうか
)

# ダミーデータを生成
seq_length = 10  # シーケンスの長さ
batch_size = 16  # バッチサイズ
input_dim = 32  # 入力の次元数
x = torch.randn(seq_length, batch_size, input_dim)

# forwardメソッドを呼び出す
output = model(x)

# 出力の形状を表示
print(output.shape)


torch.Size([10, 16, 32])


In [1]:
import torch
import torch.nn as nn
from typing import Optional, Tuple

class _ConvolutionModule(torch.nn.Module):
    r"""LE_Conformer convolution module.

    Args:
        input_dim (int): input dimension.
        num_channels (int): number of depthwise convolution layer input channels.
        depthwise_kernel_size (int): kernel size of depthwise convolution layer.
        dropout (float, optional): dropout probability. (Default: 0.0)
        bias (bool, optional): indicates whether to add bias term to each convolution layer. (Default: ``False``)
        use_group_norm (bool, optional): use GroupNorm rather than BatchNorm. (Default: ``False``)
    """

    def __init__(
        self,
        input_dim: int,  # 入力の次元数
        num_channels: int,  # 畳み込み層のチャンネル数
        depthwise_kernel_size: int,  # 深さ方向の畳み込みのカーネルサイズ
        dropout: float = 0.0,  # ドロップアウトの割合
        bias: bool = False,  # バイアス項を使用するかどうか
        use_group_norm: bool = False,  # GroupNormを使用するかどうか
    ) -> None:
        super().__init__()
        # depthwise_kernel_sizeが奇数でなければならない理由は、'SAME'パディングを実現するためです。
        if (depthwise_kernel_size - 1) % 2 != 0:
            raise ValueError("depthwise_kernel_size must be odd to achieve 'SAME' padding.")
        # LayerNorm層
        self.layer_norm = torch.nn.LayerNorm(input_dim)
        # 畳み込み層と活性化関数を含むシーケンシャルなネットワーク
        self.sequential = torch.nn.Sequential(
            # Pointwise convolution: 入力次元から2倍のチャンネル数へ
            torch.nn.Conv1d(
                input_dim,
                2 * num_channels,
                kernel_size=1, # カーネルサイズ1はPointwiseConvの特徴
                stride=1,
                padding=0,
                bias=bias,
            ),
            # GLU活性化関数: チャンネル数を半分にする
            torch.nn.GLU(dim=1),
            # Depthwise convolution: 各チャンネル内での畳み込みを行う
            torch.nn.Conv1d(
                num_channels,
                num_channels,
                depthwise_kernel_size,
                stride=1,
                padding=(depthwise_kernel_size - 1) // 2,
                groups=num_channels,
                bias=bias,
            ),
            # GroupNormまたはBatchNorm層
            torch.nn.GroupNorm(num_groups=1, num_channels=num_channels)
            if use_group_norm
            else torch.nn.BatchNorm1d(num_channels),
            # SiLU活性化関数
            torch.nn.SiLU(),
            # Pointwise convolution: チャンネル数を元の入力次元に戻す
            torch.nn.Conv1d(
                num_channels,
                input_dim,
                kernel_size=1, # カーネルサイズ1はPointwiseConvの特徴
                stride=1,
                padding=0,
                bias=bias,
            ),
            # ドロップアウト層
            torch.nn.Dropout(dropout),
        )

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        r"""
        Args:
            input (torch.Tensor): with shape `(L,B, D)`.

        Returns:
            torch.Tensor: output, with shape `(L,B, D)`.
        """
        x = self.layer_norm(input)
        x = x.permute(1, 2,0) # 1D Convはlengthで畳み込みたい、Dはチャネルとしたいので(B,D,L)とする。
        x = self.sequential(x)
        return x.permute(2,0,1) #(L,B, D)に戻す。
class SEBlock1D(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock1D, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class Conv1DBlockWithSE(nn.Module):
    def __init__(self, in_channels, depthwise_kernel_size):
        super().__init__()
        self.layer_norm = torch.nn.LayerNorm(in_channels)
        self.conv = nn.Conv1d(in_channels, in_channels, 
                depthwise_kernel_size,
                stride=1,
                padding=(depthwise_kernel_size - 1) // 2,)
        self.bn = nn.BatchNorm1d(in_channels)
        self.swish = nn.SiLU()
        self.se = SEBlock1D(in_channels)

    def forward(self, x):
        x = self.layer_norm(x)
        x = x.permute(1, 2, 0)
        x = self.conv(x)
        x = self.bn(x)
        x = self.swish(x)
        x = self.se(x)
        x = x.permute(2, 0, 1)
        return x


class _FeedForwardModule(torch.nn.Module):# こちらは特に迷うこともなく論文そのままなのでコメントは割愛
    r"""Positionwise feed forward layer.

    Args:
        input_dim (int): input dimension.
        hidden_dim (int): hidden dimension.
        dropout (float, optional): dropout probability. (Default: 0.0)
    """

    def __init__(self, input_dim: int, hidden_dim: int, depthwise_kernel_size: int, dropout: float = 0.0) -> None:
        super().__init__()
        self.layer_norm = nn.LayerNorm(input_dim)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.activation = Conv1DBlockWithSE(hidden_dim,depthwise_kernel_size)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, input_dim)

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        input = self.layer_norm(input)
        hidden = self.fc1(input)
        hidden = self.activation(hidden)
        hidden = self.dropout(hidden)
        output = self.fc2(hidden)
        return output

class LE_ConformerLayer(torch.nn.Module):
    r"""LE_Conformer layer that constitutes LE_Conformer.

    Args:
        input_dim (int): input dimension.
        ffn_dim (int): hidden layer dimension of feedforward network.
        num_attention_heads (int): number of attention heads.
        depthwise_conv_kernel_size (int): kernel size of depthwise convolution layer.
        dropout (float, optional): dropout probability. (Default: 0.0)
        use_group_norm (bool, optional): use ``GroupNorm`` rather than ``BatchNorm1d``
            in the convolution module. (Default: ``False``)
        convolution_first (bool, optional): apply the convolution module ahead of
            the attention module. (Default: ``False``)
    """

    def __init__(
        self,
        input_dim: int,
        ffn_dim: int,
        num_attention_heads: int,
        depthwise_conv_kernel_size: int,
        dropout: float = 0.0,
        use_group_norm: bool = False,
        convolution_first: bool = False,
    ) -> None:
        super().__init__()

        # 最初のフィードフォワードネットワークを定義します。このネットワークは、入力データを処理し、
        # 隠れ層の次元をffn_dimに変換します。
        self.ffn1 = _FeedForwardModule(input_dim, ffn_dim, depthwise_conv_kernel_size, dropout=dropout)

        # レイヤーノーマライゼーションを定義します。これは、入力データのスケールを正規化するために使用されます。
        self.self_attn_layer_norm = torch.nn.LayerNorm(input_dim)
        # セルフアテンションメカニズムを定義します。これは、入力データの各部分が他の部分とどの程度関連しているかを学習します。
        self.self_attn = torch.nn.MultiheadAttention(input_dim, num_attention_heads, dropout=dropout)
        # ドロップアウトを定義します。これは、ネットワークの過学習を防ぐために使用されます。
        self.self_attn_dropout = torch.nn.Dropout(dropout)

        # 畳み込みモジュールを定義します。これは、入力データに畳み込みを適用し、特徴を抽出します。
        self.conv_module = _ConvolutionModule(
            input_dim=input_dim,
            num_channels=input_dim,
            depthwise_kernel_size=depthwise_conv_kernel_size,
            dropout=dropout,
            bias=True,
            use_group_norm=use_group_norm,
        )

        # 二番目のフィードフォワードネットワークを定義します。このネットワークは、畳み込みモジュールとセルフアテンションメカニズムによって
        # 処理し、出力の次元をinput_dimに戻します。
        self.ffn2 = _FeedForwardModule(input_dim, ffn_dim, depthwise_conv_kernel_size, dropout=dropout)
        # 最終的なレイヤーノーマライゼーションを定義します。これは、出力データのスケールを正規化するために使用されます。
        self.final_layer_norm = torch.nn.LayerNorm(input_dim)
        # 畳み込みモジュールをセルフアテンションメカニズムの前に適用するかどうかを決定します。
        self.convolution_first = convolution_first

    def _apply_convolution(self, input: torch.Tensor) -> torch.Tensor:
        # 入力データを保存します。これは後で残差接続を作成するために使用されます。
        residual = input
        # 入力データの次元を変更します。これは、畳み込みモジュールが適切に機能するために必要です。
        input = input.transpose(0, 1)
        # 畳み込みモジュールを適用します。これにより、入力データに畳み込みが適用され、特徴が抽出されます。
        input = self.conv_module(input)
        # 入力データの次元を元に戻します。これは、後続の処理ステップが適切に機能するために必要です。
        input = input.transpose(0, 1)
        # 残差接続を作成します。これは、ネットワークが入力データの重要な情報を保持できるようにするために使用されます。
        input = residual + input
        return input

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        r"""
        Args:
            input (torch.Tensor): input, with shape `(T, B, D)`.
            key_padding_mask (torch.Tensor or None): key padding mask to use in self attention layer.

        Returns:
            torch.Tensor: output, with shape `(T, B, D)`.
        """
        # 入力データを保存します。これは後で残差接続を作成するために使用されます。
        residual = input
        # 最初のフィードフォワードネットワークを適用します。これにより、入力データが処理され、隠れ層の次元がffn_dimに変換されます。
        x = self.ffn1(input)
        # スケールされた残差接続を作成します。これは、ネットワークが入力データの重要な情報を保持できるようにするために使用されます。
        x = x * 0.5 + residual

        # 畳み込みモジュールを最初に適用する場合、ここでそれを行います。
        if self.convolution_first:
            x = self._apply_convolution(x)

        # 入力データを保存します。これは後で残差接続を作成するために使用されます。
        residual = x
        # レイヤーノーマライゼーションを適用します。これは、入力データのスケールを正規化するために使用されます。
        x = self.self_attn_layer_norm(x)
        # セルフアテンションメカニズムを適用します。これは、入力データの各部分が他の部分とどの程度関連しているかを学習します。
        x, _ = self.self_attn(
            query=x,
            key=x,
            value=x,
            need_weights=False,
            
        )
        # ドロップアウトを適用します。これは、ネットワークの過学習を防ぐために使用されます。
        x = self.self_attn_dropout(x)
        # 残差接続を作成します。これは、ネットワークが入力データの重要な情報を保持できるようにするために使用されます。
        x = x + residual

        # 畳み込みモジュールを最初に適用しなかった場合、ここでそれを行います。
        if not self.convolution_first:
            x = self._apply_convolution(x)

        # 入力データを保存します。これは後で残差接続を作成するために使用されます。
        residual = x
        # 二番目のフィードフォワードネットワークを適用します。これにより、畳み込みモジュールとセルフアテンションメカニズムによって
        # 処理されたデータが処理され、出力の次元がinput_dimに戻されます。
        x = self.ffn2(x)
        # スケールされた残差接続を作成します。これは、ネットワークが入力データの重要な情報を保持できるようにするために使用されます。
        x = x * 0.5 + residual

        # 最終的なレイヤーノーマライゼーションを適用します。これは、出力データのスケールを正規化するために使用されます。
        x = self.final_layer_norm(x)
        # 処理されたデータを返します。
        return x

class LE_Conformer(torch.nn.Module):
    def __init__(
        self,
        input_dim: int,
        num_heads: int,
        ffn_dim: int,
        num_layers: int,
        depthwise_conv_kernel_size: int,
        dropout: float = 0.0,
        use_group_norm: bool = False,
        convolution_first: bool = False,
    ):
        super().__init__()

        self.LE_conformer_layers = torch.nn.ModuleList(
            [
                LE_ConformerLayer(
                    input_dim,
                    ffn_dim,
                    num_heads,
                    depthwise_conv_kernel_size,
                    dropout=dropout,
                    use_group_norm=use_group_norm,
                    convolution_first=convolution_first,
                )
                for _ in range(num_layers)
            ]
        )

    def forward(self, input: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:


        x = input
        for layer in self.LE_conformer_layers:
            x = layer(x)
        return x